In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
model = init_chat_model(model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        model_provider="groq",
                        temperature = 0,
                        max_retries=3,
                        timeout=60,
                        max_tokens=300
                        )

In [3]:
from dataclasses import dataclass
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dataclass
class LanguageContext:
    user_language: str = "English"

@dynamic_prompt
def user_language_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_language = request.runtime.context.user_language
    base_prompt = "You are a helpful assistant."

    if user_language != "English":
        return f"{base_prompt} only respond in {user_language}."
    elif user_language == "English":
        return base_prompt

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    context_schema=LanguageContext,
    middleware=[user_language_prompt]
)

In [5]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Arabic")
)

print(response["messages"][-1].content)

مرحبًا! كيف يمكنني مساعدتك اليوم؟


In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="Spanish")
)

print(response["messages"][-1].content)

¡Claro! Estoy listo para ayudarte. ¿En qué puedo asistirte hoy?


In [7]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"message": [HumanMessage(content="Hello, how are you?")]},
    context=LanguageContext(user_language="French")
)

print(response["messages"][-1].content)

Bien sûr, je suis là pour vous aider. Quelle est votre question ou votre problème ?
